# PARTE 1: CARGA DE DATOS (ETL)




In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import logging

# Configurar el sistema de registro (logging)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("proceso_etl.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("etl_educativo")

class ProcesoETLEducativo:
    """
    Clase ETL para procesamiento de datos educativos desde varios archivos CSV.
    Mantiene las filas de totales para análisis posterior.
    """
    
    def __init__(self, dir_entrada=".", dir_salida="./datos_procesados"):
        """
        Inicializa el proceso ETL con directorios de entrada y salida.
        """
        self.dir_entrada = Path(dir_entrada)
        self.dir_salida = Path(dir_salida)
        
        # Crear directorio de salida si no existe
        if not self.dir_salida.exists():
            self.dir_salida.mkdir(parents=True)
            
        # Diccionario para almacenar dataframes
        self.dataframes = {}
        
        # Mapeo de archivos de entrada
        self.tipos_archivos = {
            'porcentajes_basico.csv': 'porcentaje_basico',
            'porcentajes_medio.csv': 'porcentaje_medio',
            'porcentajes_superior.csv': 'porcentaje_superior',
            'ciclos_formativos_basico.csv': 'ciclo_basico',
            'ciclos_formativos_medio.csv': 'ciclo_medio',
            'ciclos_formativos_superior.csv': 'ciclo_superior',
            'cursos_especializacion.csv': 'cursos_especializacion',
            'Cursos_alumnos_notas.xlsx': 'cursos_notas'
        }
        
        logger.info("Proceso ETL inicializado")
    
    def extraer(self):
        """
        Extrae datos de archivos CSV y Excel en el directorio de entrada.
        """
        logger.info("Iniciando fase de extracción")
        
        for nombre_archivo, tipo_archivo in self.tipos_archivos.items():
            try:
                if nombre_archivo.endswith('.xlsx') or nombre_archivo.endswith('.xls'):
                    # Leer archivo Excel
                    df = pd.read_excel(nombre_archivo, sheet_name=0)
                    logger.info(f"Lectura exitosa del archivo Excel {nombre_archivo} con {len(df)} filas")
                else:
                    # Leer archivo CSV
                    df = pd.read_csv(
                        nombre_archivo,
                        delimiter=';',
                        encoding='utf-8'
                    )
                    logger.info(f"Lectura exitosa de {nombre_archivo} con {len(df)} filas")
                    logger.info(f"Columnas originales: {df.columns.tolist()}")
                
                self.dataframes[tipo_archivo] = df
                
            except FileNotFoundError:
                logger.warning(f"Archivo {nombre_archivo} no encontrado")
            except Exception as e:
                logger.error(f"Error extrayendo datos de {nombre_archivo}: {str(e)}")
                
        logger.info(f"Extracción completa. Cargados {len(self.dataframes)} archivos")
        return self
    
    def transformar(self):
        """
        Transforma los datos extraídos.
        NOTA: Ahora mantiene las filas de totales en cursos de especialización.
        """
        logger.info("Iniciando fase de transformación")
        
        for tipo_archivo, df in list(self.dataframes.items()):
            logger.info(f"Transformando {tipo_archivo}")
            
            # Limpiar nombres de columnas (eliminar notas entre paréntesis)
            df.columns = [col.split(' (')[0].strip() for col in df.columns]
            
            # Procesar archivo de cursos de especialización
            if tipo_archivo == 'cursos_especializacion':
                logger.info(f"Procesando cursos de especialización con columnas: {df.columns.tolist()}")
                
                # Estandarizar nombre de columna si es necesario
                if 'Familia profesional/curso especialización' in df.columns:
                    df = df.rename(columns={'Familia profesional/curso especialización': 'Familia profesional'})
                    logger.info("Renombrada columna a 'Familia profesional'")
                
                # NUEVO: Añadir columna indicadora para identificar totales
                if 'Familia profesional' in df.columns:
                    df['es_total'] = df['Familia profesional'].str.contains('TOTAL', case=False, na=False)
                    
                    # Contar cuántas filas son totales
                    num_totales = df['es_total'].sum()
                    logger.info(f"Se identificaron {num_totales} filas de totales en cursos de especialización")
                    
                    # Mostrar las filas de totales identificadas
                    filas_totales = df[df['es_total']]['Familia profesional'].unique()
                    logger.info(f"Tipos de totales encontrados: {filas_totales}")
                    
                    # NO ELIMINAMOS las filas de totales, solo las marcamos
                    logger.info("Manteniendo filas de totales para análisis posterior")
            
            # Estandarizar nombres de columnas para porcentajes
            elif tipo_archivo.startswith('porcentaje_'):
                # Renombrar columnas para que sean consistentes
                rename_map = {}
                
                # Buscar y estandarizar columnas
                for col in df.columns:
                    if 'Comunidad autónoma' in col:
                        rename_map[col] = 'Comunidad autónoma'
                    elif 'Familia profesional' in col:
                        rename_map[col] = 'Familia profesional'
                    elif col == 'Total':
                        # Cambiar 'Total' a 'Porcentajes total de módulos aprobados'
                        rename_map[col] = 'Porcentajes total de módulos aprobados'
                
                if rename_map:
                    df = df.rename(columns=rename_map)
                    logger.info(f"Columnas renombradas: {rename_map}")
                
                logger.info(f"Columnas después de estandarización: {df.columns.tolist()}")
            
            # Eliminar filas de Cataluña
            if 'Comunidad autónoma' in df.columns:
                filas_antes = len(df)
                df = df[df['Comunidad autónoma'] != 'Cataluña'].copy()
                filas_eliminadas = filas_antes - len(df)
                logger.info(f"Se eliminaron {filas_eliminadas} filas de Cataluña en {tipo_archivo}")
                
                # Eliminar filas especiales que no son comunidades autónomas reales
                filas_especiales = [
                    'Porcentajes total de módulos aprobados',
                    'TOTAL',
                    'Total',
                    'Promedio',
                    'Media',
                    'Porcentaje total'
                ]
                
                filas_antes = len(df)
                df = df[~df['Comunidad autónoma'].isin(filas_especiales)].copy()
                filas_eliminadas = filas_antes - len(df)
                logger.info(f"Se eliminaron {filas_eliminadas} filas especiales (totales/promedios) en {tipo_archivo}")
                
                # Mostrar las comunidades autónomas únicas después de la limpieza
                comunidades_unicas = df['Comunidad autónoma'].unique()
                logger.info(f"Comunidades autónomas después de limpieza: {len(comunidades_unicas)}")
                logger.info(f"Muestra de comunidades: {comunidades_unicas[:5]}")
            
            # Eliminar filas especiales en ciclos formativos
            if tipo_archivo.startswith('ciclo_') and 'Ciclo formativo' in df.columns:
                # Marcar totales en ciclos formativos
                df['es_total'] = df['Ciclo formativo'].str.contains('Total|TOTAL|alumnado que terminó', case=False, regex=True, na=False)
                
                # Contar totales
                num_totales = df['es_total'].sum()
                logger.info(f"Se identificaron {num_totales} filas de totales en {tipo_archivo}")
                
                # SEGUIMOS eliminando los totales en ciclos formativos porque no aportan información estructurada
                filas_antes = len(df)
                df = df[~df['es_total']].copy()
                filas_eliminadas = filas_antes - len(df)
                logger.info(f"Se eliminaron {filas_eliminadas} filas de totales en ciclos formativos {tipo_archivo}")
                
                # Eliminar la columna es_total ya que no la necesitamos
                df = df.drop('es_total', axis=1)
                
                # Mostrar muestra de ciclos después de limpieza
                ciclos_unicos = df['Ciclo formativo'].unique()
                logger.info(f"Ciclos formativos después de limpieza: {len(ciclos_unicos)}")
                logger.info(f"Muestra de ciclos: {ciclos_unicos[:5]}")
            
            # Para familias profesionales en otros archivos, seguimos eliminando totales
            if 'Familia profesional' in df.columns and tipo_archivo != 'cursos_especializacion':
                filas_especiales_familias = [
                    'TOTAL',
                    'Total',
                    'Promedio',
                    'Media',
                    'Total general',
                    'TOTAL FAMILIAS'
                ]
                
                filas_antes = len(df)
                df = df[~df['Familia profesional'].str.upper().isin([f.upper() for f in filas_especiales_familias])].copy()
                filas_eliminadas = filas_antes - len(df)
                if filas_eliminadas > 0:
                    logger.info(f"Se eliminaron {filas_eliminadas} filas especiales en familias profesionales de {tipo_archivo}")
            
            # Añadir columna de nivel educativo para datasets de porcentaje
            if tipo_archivo.startswith('porcentaje_'):
                # Extraer el nivel del nombre del archivo
                if 'basico' in tipo_archivo:
                    nivel = 'BASICO'
                elif 'medio' in tipo_archivo:
                    nivel = 'MEDIO'
                elif 'superior' in tipo_archivo:
                    nivel = 'SUPERIOR'
                else:
                    nivel = tipo_archivo.split('_')[1].upper()
                
                df['nivel_educativo'] = nivel
                logger.info(f"Añadida columna nivel_educativo = {nivel}")
            
            # Añadir columna de nivel para datasets de ciclos
            if tipo_archivo.startswith('ciclo_'):
                if 'basico' in tipo_archivo:
                    nivel = 'BASICO'
                elif 'medio' in tipo_archivo:
                    nivel = 'MEDIO'
                elif 'superior' in tipo_archivo:
                    nivel = 'SUPERIOR'
                else:
                    nivel = tipo_archivo.split('_')[1].upper()
                
                df['nivel_ciclo'] = nivel
                logger.info(f"Añadida columna nivel_ciclo = {nivel}")
            
            # Manejar valores faltantes y conversión de tipos para columna de porcentajes
            col_porcentajes = 'Porcentajes total de módulos aprobados'
            if col_porcentajes in df.columns:
                # Convertir a string para procesar
                df[col_porcentajes] = df[col_porcentajes].astype(str)
                
                # Reemplazar puntos solos con NaN
                df.loc[df[col_porcentajes].str.strip().isin(['.', '..', '...']), col_porcentajes] = np.nan
                
                # Para porcentajes: convertir coma decimal a punto
                if tipo_archivo.startswith('porcentaje_'):
                    df[col_porcentajes] = df[col_porcentajes].apply(
                        lambda x: np.nan if pd.isna(x) or x == 'nan' else float(str(x).replace(',', '.'))
                    )
                
                logger.info(f"Estadísticas de {col_porcentajes}: min={df[col_porcentajes].min()}, max={df[col_porcentajes].max()}, media={df[col_porcentajes].mean():.2f}")
            
            # Para datos de ciclos y especialización, mantener 'Total' como está
            elif 'Total' in df.columns:
                df['Total'] = df['Total'].astype(str)
                df.loc[df['Total'].str.strip().isin(['.', '..', '...']), 'Total'] = np.nan
                df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
            
            # Limpiar columnas de texto
            for col in df.select_dtypes(include=['object']).columns:
                if col not in ['nivel_educativo', 'nivel_ciclo', 'es_total']:
                    df[col] = df[col].str.strip()
            
            # Actualizar el dataframe en nuestro diccionario
            self.dataframes[tipo_archivo] = df
            
            logger.info(f"Transformación completa para {tipo_archivo}")
        
        # Crear dataset combinado de porcentajes
        self._crear_dataset_combinado_porcentajes()
        
        # Crear dataset combinado de ciclos
        self._crear_dataset_combinado_ciclos()
        
        logger.info("Fase de transformación completa")
        return self
    
    def _crear_dataset_combinado_porcentajes(self):
        """
        Crea un dataset combinado con todos los porcentajes
        """
        try:
            dfs_porcentaje = []
            for tipo_archivo, df in self.dataframes.items():
                if tipo_archivo.startswith('porcentaje_'):
                    logger.info(f"Añadiendo {tipo_archivo} al dataset combinado")
                    logger.info(f"  Columnas: {df.columns.tolist()}")
                    logger.info(f"  Nivel: {df['nivel_educativo'].iloc[0] if 'nivel_educativo' in df.columns else 'N/A'}")
                    logger.info(f"  Filas: {len(df)}")
                    
                    # Verificar que las columnas necesarias existen
                    columnas_requeridas = ['Sexo', 'Comunidad autónoma', 'Familia profesional', 'Porcentajes total de módulos aprobados', 'nivel_educativo']
                    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
                    if columnas_faltantes:
                        logger.warning(f"  Columnas faltantes en {tipo_archivo}: {columnas_faltantes}")
                    
                    dfs_porcentaje.append(df)
            
            if dfs_porcentaje:
                df_combinado = pd.concat(dfs_porcentaje, ignore_index=True)
                self.dataframes['todos_porcentajes'] = df_combinado
                
                # Verificar que tenemos los tres niveles
                niveles_unicos = df_combinado['nivel_educativo'].unique()
                logger.info(f"Niveles educativos en dataset combinado: {niveles_unicos}")
                logger.info(f"Creado dataset combinado de porcentajes con {len(df_combinado)} filas")
                
                # Verificar estadísticas por nivel
                col_porcentajes = 'Porcentajes total de módulos aprobados'
                for nivel in niveles_unicos:
                    df_nivel = df_combinado[df_combinado['nivel_educativo'] == nivel]
                    logger.info(f"  {nivel}: {len(df_nivel)} filas, {col_porcentajes} media={df_nivel[col_porcentajes].mean():.2f}")
                
                # Mostrar comunidades autónomas únicas en el dataset combinado
                comunidades_combinado = df_combinado['Comunidad autónoma'].unique()
                logger.info(f"Comunidades autónomas en dataset combinado: {len(comunidades_combinado)}")
                logger.info(f"Muestra: {comunidades_combinado[:10]}")
                
        except Exception as e:
            logger.error(f"Error creando dataset combinado de porcentajes: {str(e)}")
            import traceback
            logger.error(traceback.format_exc())
    
    def _crear_dataset_combinado_ciclos(self):
        """
        Crea un dataset combinado con todos los ciclos
        """
        try:
            dfs_ciclo = []
            for tipo_archivo, df in self.dataframes.items():
                if tipo_archivo.startswith('ciclo_'):
                    logger.info(f"Añadiendo {tipo_archivo} al dataset combinado de ciclos")
                    
                    # Verificar que no haya filas especiales
                    if 'Ciclo formativo' in df.columns:
                        ciclos_unicos = df['Ciclo formativo'].unique()
                        logger.info(f"  Ciclos en {tipo_archivo}: {len(ciclos_unicos)}")
                        logger.info(f"  Muestra: {ciclos_unicos[:5]}")
                    
                    dfs_ciclo.append(df)
            
            if dfs_ciclo:
                df_combinado = pd.concat(dfs_ciclo, ignore_index=True)
                self.dataframes['todos_ciclos'] = df_combinado
                logger.info(f"Creado dataset combinado de ciclos con {len(df_combinado)} filas")
                
                # Verificar ciclos en dataset combinado
                if 'Ciclo formativo' in df_combinado.columns:
                    ciclos_combinado = df_combinado['Ciclo formativo'].unique()
                    logger.info(f"Ciclos formativos en dataset combinado: {len(ciclos_combinado)}")
                    logger.info(f"Muestra: {ciclos_combinado[:10]}")
                    
        except Exception as e:
            logger.error(f"Error creando dataset combinado de ciclos: {str(e)}")
    
    def cargar(self, formato_archivo='csv'):
        """
        Carga datos transformados a archivos de salida.
        """
        logger.info(f"Iniciando fase de carga, guardando como {formato_archivo}")
        
        # Asegurar que el directorio de salida existe
        if not self.dir_salida.exists():
            self.dir_salida.mkdir(parents=True)
            
        for tipo_archivo, df in self.dataframes.items():
            try:
                ruta_salida = self.dir_salida / f"{tipo_archivo}_procesado.csv"
                df.to_csv(ruta_salida, index=False, encoding='utf-8')
                logger.info(f"Guardado {tipo_archivo} en {ruta_salida}")
                
                # Verificación adicional para cursos de especialización
                if tipo_archivo == 'cursos_especializacion':
                    if 'Familia profesional' in df.columns and 'es_total' in df.columns:
                        # Mostrar información sobre totales guardados
                        totales = df[df['es_total']]['Familia profesional'].unique()
                        no_totales = df[~df['es_total']]['Familia profesional'].unique()
                        
                        logger.info(f"Cursos de especialización guardados:")
                        logger.info(f"  - Filas de totales: {len(df[df['es_total']])}")
                        logger.info(f"  - Filas de familias reales: {len(df[~df['es_total']])}")
                        logger.info(f"  - Tipos de totales: {totales}")
                        logger.info(f"  - Muestra de familias reales: {no_totales[:5]}")
                
            except Exception as e:
                logger.error(f"Error guardando {tipo_archivo}: {str(e)}")
                
        logger.info("Fase de carga completa")
        return self
    
    def ejecutar_pipeline(self, formato_archivo='csv'):
        """
        Ejecuta el pipeline ETL completo: extraer, transformar y cargar.
        """
        logger.info("Iniciando pipeline ETL")
        
        try:
            self.extraer()
            self.transformar()
            self.cargar(formato_archivo)
            logger.info("Pipeline ETL completado exitosamente")
            return True
        except Exception as e:
            logger.error(f"Fallo en pipeline ETL: {str(e)}")
            import traceback
            logger.error(traceback.format_exc())
            return False

    def obtener_resumen_datos(self):
        """
        Genera un resumen básico de los datos procesados.
        """
        logger.info("Generando resumen de datos")
        
        resumen = {}
        for tipo_archivo, df in self.dataframes.items():
            resumen[tipo_archivo] = {
                'filas': len(df),
                'columnas': len(df.columns),
                'columnas_lista': df.columns.tolist(),
                'valores_nulos': df.isnull().sum().to_dict(),
                'tipos_datos': df.dtypes.astype(str).to_dict()
            }
            
            # Información específica sobre niveles
            if 'nivel_educativo' in df.columns:
                resumen[tipo_archivo]['niveles_educativos'] = df['nivel_educativo'].unique().tolist()
            if 'nivel_ciclo' in df.columns:
                resumen[tipo_archivo]['niveles_ciclo'] = df['nivel_ciclo'].unique().tolist()
            
            # Información sobre comunidades autónomas
            if 'Comunidad autónoma' in df.columns:
                resumen[tipo_archivo]['comunidades_autonomas'] = df['Comunidad autónoma'].unique().tolist()
                resumen[tipo_archivo]['num_comunidades'] = len(df['Comunidad autónoma'].unique())
            
            # Información sobre ciclos formativos
            if 'Ciclo formativo' in df.columns:
                resumen[tipo_archivo]['ciclos_formativos'] = df['Ciclo formativo'].unique().tolist()[:10]
                resumen[tipo_archivo]['num_ciclos'] = len(df['Ciclo formativo'].unique())
            
            # Información sobre familias profesionales
            if 'Familia profesional' in df.columns:
                resumen[tipo_archivo]['familias_profesionales'] = df['Familia profesional'].unique().tolist()
                resumen[tipo_archivo]['num_familias'] = len(df['Familia profesional'].unique())
                
                # Verificar si tenemos columna es_total
                if 'es_total' in df.columns:
                    resumen[tipo_archivo]['tiene_totales'] = True
                    resumen[tipo_archivo]['num_totales'] = df['es_total'].sum()
                    resumen[tipo_archivo]['num_familias_reales'] = len(df[~df['es_total']]['Familia profesional'].unique())
                else:
                    resumen[tipo_archivo]['tiene_totales'] = False
            
            # Estadísticas básicas para columna de porcentajes si existe
            col_porcentajes = 'Porcentajes total de módulos aprobados'
            if col_porcentajes in df.columns:
                resumen[tipo_archivo]['estadisticas_porcentajes'] = {
                    'media': df[col_porcentajes].mean(),
                    'mediana': df[col_porcentajes].median(),
                    'min': df[col_porcentajes].min(),
                    'max': df[col_porcentajes].max(),
                    'nulos': df[col_porcentajes].isnull().sum()
                }
        
        return resumen


# Ejemplo de uso
if __name__ == "__main__":
    # Crear una instancia de la clase ETL
    etl = ProcesoETLEducativo(dir_entrada=".", dir_salida="./datos_procesados")
    
    # Ejecutar el ETL
    exito = etl.ejecutar_pipeline()
    
    if exito:
        # Mostrar resumen de datos
        resumen = etl.obtener_resumen_datos()
        for tipo_archivo, info in resumen.items():
            print(f"\nResumen para {tipo_archivo}:")
            print(f"  Filas: {info['filas']}")
            print(f"  Columnas: {info['columnas']}")
            
            if 'tiene_totales' in info and info['tiene_totales']:
                print(f"  Filas de totales: {info['num_totales']}")
                print(f"  Familias profesionales reales: {info['num_familias_reales']}")
            
            if 'num_comunidades' in info:
                print(f"  Número de comunidades: {info['num_comunidades']}")
            if 'num_ciclos' in info:
                print(f"  Número de ciclos formativos: {info['num_ciclos']}")
            if 'num_familias' in info:
                print(f"  Número de familias profesionales (incluyendo totales): {info['num_familias']}")
    else:
        print("Proceso ETL falló. Revise los logs para más detalles.")

2025-05-11 18:38:48,750 - etl_educativo - INFO - Proceso ETL inicializado
2025-05-11 18:38:48,753 - etl_educativo - INFO - Iniciando pipeline ETL
2025-05-11 18:38:48,754 - etl_educativo - INFO - Iniciando fase de extracción
2025-05-11 18:38:48,765 - etl_educativo - INFO - Lectura exitosa de porcentajes_basico.csv con 1560 filas
2025-05-11 18:38:48,765 - etl_educativo - INFO - Columnas originales: ['Sexo', 'Comunidad autónoma', 'Familia profesional', 'Total']
2025-05-11 18:38:48,772 - etl_educativo - INFO - Lectura exitosa de porcentajes_medio.csv con 1560 filas
2025-05-11 18:38:48,773 - etl_educativo - INFO - Columnas originales: ['Sexo', 'Comunidad autónoma (1)', 'Familia profesional (2)', 'Total']
2025-05-11 18:38:48,779 - etl_educativo - INFO - Lectura exitosa de porcentajes_superior.csv con 1560 filas
2025-05-11 18:38:48,780 - etl_educativo - INFO - Columnas originales: ['Sexo', 'Comunidad autónoma (1)', 'Familia profesional (2)', 'Porcentajes total de módulos aprobados']
2025-05-1


Resumen para porcentaje_basico:
  Filas: 1404
  Columnas: 5
  Número de comunidades: 18
  Número de familias profesionales (incluyendo totales): 26

Resumen para porcentaje_medio:
  Filas: 1404
  Columnas: 5
  Número de comunidades: 18
  Número de familias profesionales (incluyendo totales): 26

Resumen para porcentaje_superior:
  Filas: 1404
  Columnas: 5
  Número de comunidades: 18
  Número de familias profesionales (incluyendo totales): 26

Resumen para ciclo_basico:
  Filas: 8883
  Columnas: 5
  Número de ciclos formativos: 47

Resumen para ciclo_medio:
  Filas: 16254
  Columnas: 5
  Número de ciclos formativos: 86

Resumen para ciclo_superior:
  Filas: 22113
  Columnas: 5
  Número de ciclos formativos: 117

Resumen para cursos_especializacion:
  Filas: 135
  Columnas: 5
  Filas de totales: 18
  Familias profesionales reales: 13
  Número de familias profesionales (incluyendo totales): 15

Resumen para cursos_notas:
  Filas: 83504
  Columnas: 4

Resumen para todos_porcentajes:
  Fi

# PARTE 2: Análisis estadístico y descriptivo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar visualización
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Directorio donde están los datos procesados
dir_datos = "./datos_procesados"
dir_resultados = os.path.join(dir_datos, "eda_resultados")

# Asegurar que existe el directorio para resultados
if not os.path.exists(dir_resultados):
    os.makedirs(dir_resultados)

def cargar_datos_procesados():
    """
    Carga los datos procesados desde archivos CSV (excluyendo cursos_notas)
    """
    datos = {}
    
    archivos = [
        'porcentaje_basico_procesado.csv',
        'porcentaje_medio_procesado.csv',
        'porcentaje_superior_procesado.csv',
        'ciclo_basico_procesado.csv',
        'ciclo_medio_procesado.csv',
        'ciclo_superior_procesado.csv',
        'cursos_especializacion_procesado.csv',
        'todos_porcentajes_procesado.csv',
        'todos_ciclos_procesado.csv'
    ]
    
    for archivo in archivos:
        ruta_completa = os.path.join(dir_datos, archivo)
        if os.path.exists(ruta_completa):
            nombre_simple = archivo.replace('_procesado.csv', '')
            try:
                datos[nombre_simple] = pd.read_csv(ruta_completa)
                print(f"Cargado: {archivo} con {len(datos[nombre_simple])} filas")
                # Mostrar si tiene columna es_total
                if 'es_total' in datos[nombre_simple].columns:
                    num_totales = datos[nombre_simple]['es_total'].sum()
                    print(f"  Contiene {num_totales} filas de totales")
            except Exception as e:
                print(f"Error cargando {archivo}: {e}")
    
    return datos

def analisis_cursos_especializacion(datos, dir_salida):
    """
    Análisis de cursos de especialización
    """
    print("\n=== ANÁLISIS DE CURSOS DE ESPECIALIZACIÓN ===")
    
    if 'cursos_especializacion' not in datos:
        return
    
    df = datos['cursos_especializacion']
    
    # Verificar estructura del archivo
    print(f"Columnas disponibles: {df.columns.tolist()}")
    
    # Verificar si tenemos la columna es_total
    if 'es_total' in df.columns:
        print(f"Total de filas: {len(df)}")
        print(f"Filas de totales: {df['es_total'].sum()}")
        print(f"Filas de familias reales: {(~df['es_total']).sum()}")
        
        # Para análisis de familias profesionales, filtrar totales
        df_familias = df[~df['es_total']].copy()
        
        # También guardar los totales para análisis separado
        df_totales = df[df['es_total']].copy()
        
        print("\nTotales encontrados:")
        if len(df_totales) > 0:
            print(df_totales[['Familia profesional', 'Total']].groupby('Familia profesional').sum())
    else:
        # Si no hay columna es_total, asumimos que es un archivo antiguo sin esa información
        print("No se encontró columna 'es_total'. Procesando sin filtrado.")
        df_familias = df.copy()
        df_totales = pd.DataFrame()  # DataFrame vacío
    
    if 'Total' in df_familias.columns:
        # 1. Por familia profesional (solo familias reales)
        if 'Familia profesional' in df_familias.columns:
            total_familia = df_familias.groupby('Familia profesional')['Total'].sum().reset_index()
            total_familia = total_familia.sort_values('Total', ascending=False)
            
            # Visualización Top 10 familias profesionales
            plt.figure(figsize=(12, 8))
            top_familias = total_familia.head(10)
            sns.barplot(data=top_familias, x='Total', y='Familia profesional')
            plt.title('Top 10 Familias Profesionales por número de estudiantes\n(Cursos de Especialización)')
            plt.xlabel('Total de estudiantes')
            plt.tight_layout()
            plt.savefig(os.path.join(dir_salida, 'top_familias_especializacion.png'), dpi=300, bbox_inches='tight')
            plt.close()
            
            # Distribución por sexo
            if 'Sexo' in df_familias.columns:
                sexo_dist = df_familias.groupby(['Familia profesional', 'Sexo'])['Total'].sum().unstack(fill_value=0)
                
                # Top 10 familias para visualización
                top_fam_names = total_familia.head(10)['Familia profesional'].tolist()
                sexo_dist_top = sexo_dist.loc[top_fam_names]
                
                plt.figure(figsize=(14, 8))
                sexo_dist_top.plot(kind='bar', stacked=True)
                plt.title('Distribución por Sexo en Top 10 Familias Profesionales\n(Cursos de Especialización)')
                plt.xlabel('Familia Profesional')
                plt.ylabel('Número de estudiantes')
                plt.xticks(rotation=45, ha='right')
                plt.legend(title='Sexo')
                plt.tight_layout()
                plt.savefig(os.path.join(dir_salida, 'distribucion_sexo_especializacion.png'), dpi=300, bbox_inches='tight')
                plt.close()
                
                # Análisis de proporción por sexo
                sexo_prop = sexo_dist.div(sexo_dist.sum(axis=1), axis=0) * 100
                sexo_prop_top = sexo_prop.loc[top_fam_names]
                
                plt.figure(figsize=(14, 8))
                sexo_prop_top.plot(kind='bar', stacked=True)
                plt.title('Proporción por Sexo en Top 10 Familias Profesionales\n(Cursos de Especialización)')
                plt.xlabel('Familia Profesional')
                plt.ylabel('Porcentaje (%)')
                plt.xticks(rotation=45, ha='right')
                plt.legend(title='Sexo')
                plt.tight_layout()
                plt.savefig(os.path.join(dir_salida, 'proporcion_sexo_especializacion.png'), dpi=300, bbox_inches='tight')
                plt.close()
                
        # 2. Por titularidad del centro
        if 'Titularidad del centro' in df_familias.columns:
            titularidad_dist = df_familias.groupby('Titularidad del centro')['Total'].sum().reset_index()
            
            plt.figure(figsize=(10, 6))
            plt.pie(titularidad_dist['Total'], labels=titularidad_dist['Titularidad del centro'], 
                   autopct='%1.1f%%', startangle=90)
            plt.title('Distribución por Titularidad del Centro\n(Cursos de Especialización)')
            plt.tight_layout()
            plt.savefig(os.path.join(dir_salida, 'titularidad_centro_especializacion.png'), dpi=300, bbox_inches='tight')
            plt.close()
    
    # Análisis de los totales si existen
    if len(df_totales) > 0:
        print("\n=== ANÁLISIS DE TOTALES ===")
        total_general = df_totales[df_totales['Familia profesional'].str.contains('TOTAL', case=False)]
        if len(total_general) > 0:
            print(f"Total general de estudiantes: {total_general['Total'].sum()}")
            
            # Si hay totales por tipo de centro
            totales_por_tipo = df_totales[df_totales['Familia profesional'].str.contains('centros', case=False)]
            if len(totales_por_tipo) > 0:
                plt.figure(figsize=(10, 6))
                totales_por_tipo.plot(x='Familia profesional', y='Total', kind='bar')
                plt.title('Totales por Tipo de Centro')
                plt.xlabel('Tipo de Centro')
                plt.ylabel('Total de estudiantes')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.savefig(os.path.join(dir_salida, 'totales_tipo_centro.png'), dpi=300, bbox_inches='tight')
                plt.close()
    
    return df_familias, df_totales

def analisis_porcentajes_aprobados(datos, dir_salida):
    """
    Análisis de porcentajes de módulos aprobados por niveles
    """
    print("\n=== ANÁLISIS DE PORCENTAJES DE MÓDULOS APROBADOS ===")
    
    if 'todos_porcentajes' not in datos:
        return
    
    df = datos['todos_porcentajes']
    
    # 1. Comparación entre niveles educativos
    if 'nivel_educativo' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df, x='nivel_educativo', y='Porcentajes total de módulos aprobados')
        plt.title('Distribución de Porcentajes de Aprobados por Nivel Educativo')
        plt.ylabel('Porcentaje de módulos aprobados')
        plt.xlabel('Nivel Educativo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'porcentajes_por_nivel.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Estadísticas por nivel
        stats_nivel = df.groupby('nivel_educativo')['Porcentajes total de módulos aprobados'].agg([
            'count', 'mean', 'std', 'min', 'max'
        ]).round(2)
        print("\nEstadísticas por nivel educativo:")
        print(stats_nivel)
    
    # 2. Por comunidad autónoma
    if 'Comunidad autónoma' in df.columns:
        # Media por comunidad (todas juntas)
        media_ccaa = df.groupby('Comunidad autónoma')['Porcentajes total de módulos aprobados'].mean().sort_values(ascending=False)
        
        plt.figure(figsize=(12, 8))
        media_ccaa.plot(kind='bar')
        plt.title('Media de Porcentajes de Aprobados por Comunidad Autónoma')
        plt.ylabel('Porcentaje medio de módulos aprobados')
        plt.xlabel('Comunidad Autónoma')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'porcentajes_ccaa.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Comparación por nivel y comunidad
        if 'nivel_educativo' in df.columns:
            pivot_nivel_ccaa = df.pivot_table(
                values='Porcentajes total de módulos aprobados',
                index='Comunidad autónoma',
                columns='nivel_educativo',
                aggfunc='mean'
            )
            
            plt.figure(figsize=(14, 8))
            pivot_nivel_ccaa.plot(kind='bar')
            plt.title('Porcentajes de Aprobados por Comunidad Autónoma y Nivel')
            plt.ylabel('Porcentaje medio de módulos aprobados')
            plt.xlabel('Comunidad Autónoma')
            plt.xticks(rotation=45, ha='right')
            plt.legend(title='Nivel Educativo')
            plt.tight_layout()
            plt.savefig(os.path.join(dir_salida, 'porcentajes_ccaa_nivel.png'), dpi=300, bbox_inches='tight')
            plt.close()
    
    # 3. Por familia profesional
    if 'Familia profesional' in df.columns:
        # Top 10 familias con mejores porcentajes
        media_familia = df.groupby('Familia profesional')['Porcentajes total de módulos aprobados'].mean().sort_values(ascending=False)
        
        plt.figure(figsize=(12, 8))
        media_familia.head(10).plot(kind='barh')
        plt.title('Top 10 Familias Profesionales con Mejores Porcentajes de Aprobados')
        plt.xlabel('Porcentaje medio de módulos aprobados')
        plt.ylabel('Familia Profesional')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'top_familias_porcentajes.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Bottom 10 familias
        plt.figure(figsize=(12, 8))
        media_familia.tail(10).plot(kind='barh')
        plt.title('Bottom 10 Familias Profesionales con Menores Porcentajes de Aprobados')
        plt.xlabel('Porcentaje medio de módulos aprobados')
        plt.ylabel('Familia Profesional')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'bottom_familias_porcentajes.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 4. Por sexo
    if 'Sexo' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df, x='Sexo', y='Porcentajes total de módulos aprobados', hue='nivel_educativo')
        plt.title('Distribución de Porcentajes de Aprobados por Sexo y Nivel')
        plt.ylabel('Porcentaje de módulos aprobados')
        plt.xlabel('Sexo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'porcentajes_sexo_nivel.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Estadísticas por sexo
        stats_sexo = df.groupby(['Sexo', 'nivel_educativo'])['Porcentajes total de módulos aprobados'].agg([
            'count', 'mean', 'std'
        ]).round(2)
        print("\nEstadísticas por sexo y nivel:")
        print(stats_sexo)
    
    # 5. Correlaciones y análisis avanzado
    if all(col in df.columns for col in ['nivel_educativo', 'Sexo', 'Familia profesional']):
        # Heatmap de porcentajes promedio
        pivot_heatmap = df.pivot_table(
            values='Porcentajes total de módulos aprobados',
            index='Familia profesional',
            columns=['nivel_educativo', 'Sexo'],
            aggfunc='mean'
        )
        
        # Seleccionar top 15 familias para mejor visualización
        top_15_familias = media_familia.head(15).index
        pivot_heatmap_top = pivot_heatmap.loc[pivot_heatmap.index.isin(top_15_familias)]
        
        plt.figure(figsize=(14, 10))
        sns.heatmap(pivot_heatmap_top, cmap='RdYlGn', center=df['Porcentajes total de módulos aprobados'].mean(),
                   annot=True, fmt='.1f', cbar_kws={'label': 'Porcentaje de aprobados'})
        plt.title('Heatmap de Porcentajes de Aprobados\n(Top 15 Familias Profesionales)')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'heatmap_porcentajes.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    return df

def analisis_ciclos_formativos(datos, dir_salida):
    """
    Análisis de ciclos formativos por niveles
    """
    print("\n=== ANÁLISIS DE CICLOS FORMATIVOS ===")
    
    if 'todos_ciclos' not in datos:
        return
    
    df = datos['todos_ciclos']
    
    # 1. Distribución general por nivel
    if 'nivel_ciclo' in df.columns and 'Total' in df.columns:
        total_nivel = df.groupby('nivel_ciclo')['Total'].sum()
        
        plt.figure(figsize=(10, 6))
        total_nivel.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title('Distribución de Estudiantes por Nivel de Ciclo Formativo')
        plt.ylabel('')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'distribucion_nivel_ciclo.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 2. Por familia profesional y nivel
    if all(col in df.columns for col in ['Familia profesional', 'nivel_ciclo', 'Total']):
        pivot_familia_nivel = df.pivot_table(
            values='Total',
            index='Familia profesional',
            columns='nivel_ciclo',
            aggfunc='sum',
            fill_value=0
        )
        
        # Top 10 familias por total de estudiantes
        total_familia = pivot_familia_nivel.sum(axis=1).sort_values(ascending=False)
        top_10_familias = total_familia.head(10).index
        
        plt.figure(figsize=(14, 8))
        pivot_familia_nivel.loc[top_10_familias].plot(kind='bar', stacked=True)
        plt.title('Top 10 Familias Profesionales por Total de Estudiantes\n(Por Nivel de Ciclo)')
        plt.xlabel('Familia Profesional')
        plt.ylabel('Número de estudiantes')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Nivel de Ciclo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'familias_por_nivel_ciclo.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 3. Por sexo
    if 'Sexo' in df.columns:
        sexo_nivel = df.groupby(['nivel_ciclo', 'Sexo'])['Total'].sum().unstack()
        
        plt.figure(figsize=(10, 6))
        sexo_nivel.plot(kind='bar')
        plt.title('Distribución por Sexo en cada Nivel de Ciclo Formativo')
        plt.xlabel('Nivel de Ciclo')
        plt.ylabel('Número de estudiantes')
        plt.xticks(rotation=0)
        plt.legend(title='Sexo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'sexo_por_nivel_ciclo.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Proporción por sexo
        sexo_nivel_prop = sexo_nivel.div(sexo_nivel.sum(axis=1), axis=0) * 100
        
        plt.figure(figsize=(10, 6))
        sexo_nivel_prop.plot(kind='bar', stacked=True)
        plt.title('Proporción por Sexo en cada Nivel de Ciclo Formativo')
        plt.xlabel('Nivel de Ciclo')
        plt.ylabel('Porcentaje (%)')
        plt.xticks(rotation=0)
        plt.legend(title='Sexo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'proporcion_sexo_nivel_ciclo.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 4. Por comunidad autónoma
    if 'Comunidad autónoma' in df.columns:
        ccaa_nivel = df.groupby(['Comunidad autónoma', 'nivel_ciclo'])['Total'].sum().unstack(fill_value=0)
        
        # Top 10 comunidades por total
        total_ccaa = ccaa_nivel.sum(axis=1).sort_values(ascending=False)
        top_10_ccaa = total_ccaa.head(10).index
        
        plt.figure(figsize=(14, 8))
        ccaa_nivel.loc[top_10_ccaa].plot(kind='bar', stacked=True)
        plt.title('Top 10 Comunidades Autónomas por Total de Estudiantes\n(Por Nivel de Ciclo)')
        plt.xlabel('Comunidad Autónoma')
        plt.ylabel('Número de estudiantes')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Nivel de Ciclo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'ccaa_por_nivel_ciclo.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 5. Ciclos más populares
    if 'Ciclo formativo' in df.columns:
        ciclos_populares = df.groupby('Ciclo formativo')['Total'].sum().sort_values(ascending=False)
        
        plt.figure(figsize=(12, 8))
        ciclos_populares.head(15).plot(kind='barh')
        plt.title('Top 15 Ciclos Formativos más Populares')
        plt.xlabel('Número de estudiantes')
        plt.ylabel('Ciclo Formativo')
        plt.tight_layout()
        plt.savefig(os.path.join(dir_salida, 'ciclos_populares.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        # Por nivel
        if 'nivel_ciclo' in df.columns:
            for nivel in df['nivel_ciclo'].unique():
                df_nivel = df[df['nivel_ciclo'] == nivel]
                ciclos_nivel = df_nivel.groupby('Ciclo formativo')['Total'].sum().sort_values(ascending=False)
                
                plt.figure(figsize=(12, 8))
                ciclos_nivel.head(10).plot(kind='barh')
                plt.title(f'Top 10 Ciclos Formativos más Populares - Nivel {nivel}')
                plt.xlabel('Número de estudiantes')
                plt.ylabel('Ciclo Formativo')
                plt.tight_layout()
                plt.savefig(os.path.join(dir_salida, f'ciclos_populares_{nivel.lower()}.png'), dpi=300, bbox_inches='tight')
                plt.close()
    
    return df

def generar_informe_resumen(datos, dir_salida):
    """
    Genera un informe resumen con las principales estadísticas
    """
    informe = []
    informe.append("=== INFORME RESUMEN DE ANÁLISIS EXPLORATORIO ===\n")
    
    # 1. Cursos de especialización
    if 'cursos_especializacion' in datos:
        df_espec = datos['cursos_especializacion']
        if 'es_total' in df_espec.columns:
            df_real = df_espec[~df_espec['es_total']]
            total_estudiantes = df_real['Total'].sum()
        else:
            total_estudiantes = df_espec['Total'].sum()
        
        informe.append(f"CURSOS DE ESPECIALIZACIÓN:")
        informe.append(f"- Total de estudiantes: {total_estudiantes:,}")
        if 'Familia profesional' in df_espec.columns:
            n_familias = df_espec['Familia profesional'].nunique()
            informe.append(f"- Número de familias profesionales: {n_familias}")
        informe.append("")
    
    # 2. Porcentajes de aprobados
    if 'todos_porcentajes' in datos:
        df_porc = datos['todos_porcentajes']
        media_general = df_porc['Porcentajes total de módulos aprobados'].mean()
        
        informe.append(f"PORCENTAJES DE MÓDULOS APROBADOS:")
        informe.append(f"- Media general: {media_general:.2f}%")
        
        if 'nivel_educativo' in df_porc.columns:
            for nivel in df_porc['nivel_educativo'].unique():
                media_nivel = df_porc[df_porc['nivel_educativo'] == nivel]['Porcentajes total de módulos aprobados'].mean()
                informe.append(f"- Media nivel {nivel}: {media_nivel:.2f}%")
        informe.append("")
    
    # 3. Ciclos formativos
    if 'todos_ciclos' in datos:
        df_ciclos = datos['todos_ciclos']
        total_estudiantes_ciclos = df_ciclos['Total'].sum()
        
        informe.append(f"CICLOS FORMATIVOS:")
        informe.append(f"- Total de estudiantes: {total_estudiantes_ciclos:,}")
        
        if 'nivel_ciclo' in df_ciclos.columns:
            for nivel in df_ciclos['nivel_ciclo'].unique():
                total_nivel = df_ciclos[df_ciclos['nivel_ciclo'] == nivel]['Total'].sum()
                porc_nivel = (total_nivel / total_estudiantes_ciclos) * 100
                informe.append(f"- Nivel {nivel}: {total_nivel:,} ({porc_nivel:.1f}%)")
        informe.append("")
    
    # Guardar informe
    with open(os.path.join(dir_salida, 'informe_resumen.txt'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(informe))
    
    # También mostrar en consola
    print('\n'.join(informe))
    
    return informe

def main():
    """
    Función principal que ejecuta todo el análisis EDA
    """
    # Cargar datos
    datos = cargar_datos_procesados()
    
    if not datos:
        print("Error: No se pudieron cargar los datos")
        return
    
    # Ejecutar análisis
    analisis_cursos_especializacion(datos, dir_resultados)
    analisis_porcentajes_aprobados(datos, dir_resultados)
    analisis_ciclos_formativos(datos, dir_resultados)
    
    # Informe resumen
    generar_informe_resumen(datos, dir_resultados)
    
    print(f"\nAnálisis EDA completado. Resultados guardados en: {dir_resultados}")

if __name__ == "__main__":
    main()

Cargado: porcentaje_basico_procesado.csv con 1404 filas
Cargado: porcentaje_medio_procesado.csv con 1404 filas
Cargado: porcentaje_superior_procesado.csv con 1404 filas
Cargado: ciclo_basico_procesado.csv con 8883 filas
Cargado: ciclo_medio_procesado.csv con 16254 filas
Cargado: ciclo_superior_procesado.csv con 22113 filas
Cargado: cursos_especializacion_procesado.csv con 135 filas
  Contiene 18 filas de totales
Cargado: todos_porcentajes_procesado.csv con 4212 filas
Cargado: todos_ciclos_procesado.csv con 47250 filas

=== ANÁLISIS DE CURSOS DE ESPECIALIZACIÓN ===
Columnas disponibles: ['Familia profesional', 'Titularidad del centro', 'Sexo', 'Total', 'es_total']
Total de filas: 135
Filas de totales: 18
Filas de familias reales: 117

Totales encontrados:
                                               Total
Familia profesional                                 
TOTAL CURSOS PARA TITULADOS DE GRADO MEDIO       164
TOTAL CURSOS PARA TITULADOS DE GRADO SUPERIOR   1772

=== ANÁLISIS DE TOTALE

<Figure size 1400x800 with 0 Axes>

<Figure size 1400x800 with 0 Axes>

<Figure size 1400x800 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>